In [9]:
!pip install ipywidgets widgetsnbextension pandas-profiling

  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached joblib-1.1.1-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached matplotlib-3.9.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached visions-0.7.4-py3-none-any.whl.metadata (5.9 kB)
  Using cached missingno-0.5.2-py3-none-any.whl.metadata (639 bytes)
  Using cached phik-0.12.4-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.5.2 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
os.environ['TRANSFORMERS_CACHE'] ='D:/SDK/HF_cache'

In [13]:
from datasets import load_dataset

# Загружаем датасет
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd")

# Просматриваем структуру
print(dataset)

# Пример данных
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 7993
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 793
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 9994
    })
})
{'audio': {'path': None, 'array': array([ 3.05175781e-05,  3.05175781e-05,  0.00000000e+00, ...,
       -1.09863281e-03, -7.93457031e-04, -1.52587891e-04]), 'sampling_rate': 16000}, 'transcription': 'шестнадцатая часть сезона пять сериала лемони сникет тридцать три несчастья'}


Шаг 2: Предобработка данных


In [18]:
from transformers import WhisperProcessor
import torch

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

# Функция для обработки аудио и текста
def preprocess(batch):
    # Преобразуем аудио
    audio = batch["audio"]["array"]
    input_features = processor.feature_extractor(audio, sampling_rate=16000).input_features[0]

    # Проверяем, что текст существует и не пустой
    transcription = batch["transcription"]
    if transcription is None or transcription.strip() == "":
        transcription = "<unk>"  # Заменяем пустые строки на маркер "<unk>"

    # Токенизируем текст
    labels = processor.tokenizer(text=transcription, return_tensors="pt").input_ids[0]

    # Добавляем обработанные данные в батч
    batch["input_features"] = input_features
    batch["labels"] = labels
    return batch


# Применяем предобработку к датасету
processed_dataset = dataset.map(preprocess, remove_columns=["audio", "transcription"])


Map: 100%|██████████| 9994/9994 [01:45<00:00, 94.70 examples/s] 


Шаг 4: Настройка дообучения

Теперь можно приступить к обучению модели. Вот пример настройки:

In [24]:
from transformers import WhisperForConditionalGeneration, Trainer, TrainingArguments


# Загрузка модели
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./whisper_golos",
    per_device_train_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    learning_rate=2e-5,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # Отключение отчетности в WandB
    fp16=True,  # Использование 16-битных вычислений для ускорения
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor.tokenizer,  # Указываем токенизатор для автоматической обработки
)

# Запуск обучения
trainer.train()


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

Шаг 5: Оценка модели

После обучения вы можете оценить модель на тестовом наборе:

In [14]:
# Функция для оценки качества распознавания
from datasets import load_metric
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Оценка на тестовых данных
trainer.evaluate(processed_dataset["test"], metric_key_prefix="test")


ImportError: cannot import name 'load_metric' from 'datasets' (D:\Projects\Vide-to-Text\.venv\Lib\site-packages\datasets\__init__.py)

Шаг 6: Сохранение дообученной модели

После успешного обучения сохраните модель для дальнейшего использования:

In [ ]:
model.save_pretrained("./whisper_golos_finetuned")
processor.save_pretrained("./whisper_golos_finetuned")
